Check the status of tube lines and overground
https://api.tfl.gov.uk/line/mode/tube,overground,dlr,elizabeth-line/status

29-6 added overground state

In [1]:
import requests
import json
import pprint

In [2]:
import time

In [3]:
from datetime import datetime

In [4]:
datetime.now().strftime('%Y-%m-%d %H:%M:%S')

'2018-12-31 08:40:19'

In [5]:
def print_json(json_data):
    pprint.PrettyPrinter().pprint(json_data)

In [6]:
# List of allowed lines
lines = ['Bakerloo',
 'Central',
 'Circle',
 'District',
 'DLR',
 'Elizabeth Line',
 'Hammersmith & City',
 'Jubilee',
 'London Overground',
 'Metropolitan',
 'Northern',
 'Piccadilly',
 'Victoria',
 'Waterloo & City']

In [7]:
ls

development.ipynb


In [8]:
len(lines)

14

# Basic function of API

In [9]:
#url = 'https://api.tfl.gov.uk/line/mode/tube/status'   # tube only
url = 'https://api.tfl.gov.uk/line/mode/tube,overground,dlr,elizabeth-line/status'
response = requests.get(url).json() # get the response object

In [10]:
print_json(response)

[{'$type': 'Tfl.Api.Presentation.Entities.Line, Tfl.Api.Presentation.Entities',
  'created': '2018-12-11T17:06:05.303Z',
  'crowding': {'$type': 'Tfl.Api.Presentation.Entities.Crowding, '
                        'Tfl.Api.Presentation.Entities'},
  'disruptions': [],
  'id': 'bakerloo',
  'lineStatuses': [{'$type': 'Tfl.Api.Presentation.Entities.LineStatus, '
                             'Tfl.Api.Presentation.Entities',
                    'created': '0001-01-01T00:00:00',
                    'id': 0,
                    'statusSeverity': 10,
                    'statusSeverityDescription': 'Good Service',
                    'validityPeriods': []}],
  'modeName': 'tube',
  'modified': '2018-12-11T17:06:05.303Z',
  'name': 'Bakerloo',
  'routeSections': [],
  'serviceTypes': [{'$type': 'Tfl.Api.Presentation.Entities.LineServiceTypeInfo, '
                             'Tfl.Api.Presentation.Entities',
                    'name': 'Regular',
                    'uri': '/Line/Route?ids=Baker

In [11]:
lines = [line['name'] for line in response]
lines

['Bakerloo',
 'Central',
 'Circle',
 'District',
 'DLR',
 'Hammersmith & City',
 'Jubilee',
 'London Overground',
 'Metropolitan',
 'Northern',
 'Piccadilly',
 'TfL Rail',
 'Victoria',
 'Waterloo & City']

In [12]:
data_dict = dict.fromkeys(lines)    # Create the empty dict for the data
data_dict

{'Bakerloo': None,
 'Central': None,
 'Circle': None,
 'District': None,
 'DLR': None,
 'Hammersmith & City': None,
 'Jubilee': None,
 'London Overground': None,
 'Metropolitan': None,
 'Northern': None,
 'Piccadilly': None,
 'TfL Rail': None,
 'Victoria': None,
 'Waterloo & City': None}

Main logic for parsing the data into a useful dict

In [13]:
data_dict = {}

for line in response:                     # Assign data via key
    try:
        statuses = [status['statusSeverityDescription'] for status in line['lineStatuses']]
        state = ' + '.join(sorted(set(statuses)))
    
        if state == 'Good Service':   # if good status, this is the only status returned
            reason =  'Nothing to report'
        else:
            reason = ' *** '.join([status['reason'] for status in line['lineStatuses']])

        data_dict[line['name']] = {'State': state, 'Description': reason}
        
    except:
        data_dict[line['name']] = {'State': None, 'Description': 'Error parsing API data'}
    
print_json(data_dict)

{'Bakerloo': {'Description': 'Nothing to report', 'State': 'Good Service'},
 'Central': {'Description': 'Nothing to report', 'State': 'Good Service'},
 'Circle': {'Description': 'Nothing to report', 'State': 'Good Service'},
 'DLR': {'Description': "DLR: New Year's Eve, Monday 31 December. A special "
                        'all-night service will operate. Trains will run every '
                        '15 minutes on all routes from 00:30 until 08:00 on '
                        "New Year's Day",
         'State': 'Special Service'},
 'District': {'Description': 'DISTRICT LINE: Monday 31 December, and New '
                             "Year's Day 1 January, no service between Earls "
                             'Court and Kensington Olympia',
              'State': 'Part Closure'},
 'Hammersmith & City': {'Description': 'Nothing to report',
                        'State': 'Good Service'},
 'Jubilee': {'Description': 'Nothing to report', 'State': 'Good Service'},
 'London Overgroun

In [14]:
print_json(data_dict['TfL Rail'])

{'Description': 'TFL RAIL: Monday 31 December, a reduced 30 minute service '
                'will operate between Paddington and Hayes & Harlington. Train '
                'will not stop at Acton Main Line, West Ealing or Hanwell. *** '
                'TFL RAIL: Friday 28, Saturday 29, Sunday 30, Monday 31 '
                'December and Tuesday 1 January, no service between Liverpool '
                'Street and Romford. Replacement buses operate between '
                'Stratford / Newbury Park and Romford.',
 'State': 'Part Closure + Reduced Service'}
